In [26]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(style='white')

from sklearn import linear_model

In [27]:
def make_heatmap(df):
    #create figure and axis
    fig = plt.figure(figsize=(10,8))
    ax = fig.add_subplot(111)
    
    #create mask
    mask = np.zeros_like(df.corr(), dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True

    #create color scheme
    cmap = sns.diverging_palette(220, 10, as_cmap=True)

    return sns.heatmap(df.corr(),
                       mask=mask,
                       cmap=cmap,
                       center=0,
                       vmin=-0.4,
                       vmax=0.4,
                       square=True,
                       linewidths=0.5,
                       cbar_kws={'shrink': 0.5}, ax=ax);

In [28]:
def make_df(df, columns):
    _df = pd.DataFrame(df['points'])
    _df['scaled_log_prices'] = df['scaled_log']
    for col in columns:
        _df[col] = df[col]
    return pd.DataFrame(_df)

In [29]:
def generate_wine_codes(df, province, grape, vintage, price):
    
    #log price
    log_price = int(round(np.log(price), 0))
    
    #create needed containers
    wine_codes = []
    
    # create a unique binary code for province
    for taster in df['taster_names']:    
        
        #create unique wine code for each taster 
        unique_wine_code = [] #resets code to empty list for each taster
        unique_wine_code.extend(df['provinces'][province])
        unique_wine_code.extend(df['taster_names'][taster])
        unique_wine_code.extend(df['grapes'][grape])
        unique_wine_code.extend(df['vintages'][vintage])
        unique_wine_code[-1] = log_price
          
        #create a list of unique wine codes
        wine_codes.append(unique_wine_code)
        
    return wine_codes

In [32]:
def generate_prediction(model, wine_codes):
    predictions = []
    for wine_code in wine_codes:
        products = []
        for i in range(len(wine_code)):
            products.append(wine_code[i]*model.coef_[i])
        prediction = model.intercept_ + sum(products)
        predictions.append(prediction)
    return round(np.mean(predictions), 2)